In [6]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
from selenium import webdriver

seperator = ', '
def getBandTree(band, page):
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%5B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&view_state=advanced&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)
 
def download(browser, id, i = 0):
    #browser = webdriver.Chrome()
    browser.get("https://tabs.ultimate-guitar.com/download/public/" + id)
    time.sleep(1)
       
    if i == 0:    
        #click info
        doc = browser.find_elements_by_xpath('/html/body/div[3]/div/div/div/div/footer/button[1]')[0]
        doc.click()
    
    doc = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/article/div[1]/section[2]/form/div/button')[0]
    doc.click()
    #browser.quit()  

    
def main():
    band = 'iron maiden'
    page = 1
    bandURL = getBandTree(band, page)
    ids = []
    #Get max pages
    pages = bandURL.find_all('div', { "class" : "paging" })
    maxPages = str(pages).count('')
    dfs = []
    for page in range(1, maxPages + 1):
        soup = getBandTree(band, page)
        #print(bandURL)
        #mybs = soup.find_all('script')
        #print(mybs[7])
        pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
        script = soup.find("script", text=pattern)

        data = pattern.search(script.text).group(1)
        data = json.loads(data)
        for row in data["data"]["results"]: #type = Pro
            #if 'ktype' in row.keys():
            pos = row["tab_url"].find("guitar_pro_")
            if pos > 0:
                id = row["tab_url"][pos:].replace('guitar_pro_', '')
                ids.append(id)
                #download(id)
    

    browser = webdriver.Chrome()
    
    i = 0
    for id in ids:
        print(id)
        download(browser, id, i)
        i = i + 1
    browser.quit()
        
if __name__=='__main__':
    main()
    
#/artist/slipknot_505

222083


IndexError: list index out of range

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
import string
import csv
import time
import random 

def getBandList(letter, page):
    theURL = 'https://www.ultimate-guitar.com/bands/' + letter + str(page) +'.htm'
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)

def getList(file):
    letters = list(string.ascii_lowercase)
    letters .append('0-9')
    random.shuffle(letters)
    print(letters)
    page = 1
    results = []
    for letter in letters:
        time.sleep(30)
        time.sleep(random.randint(0, 25) / 3)
        print("Proccesing: " + letter)
        for page in range(1, 200):   
            time.sleep(12)
            time.sleep(random.randint(0, 6) / 3)
            print("page " + str(page))
            soup = getBandList(letter, page)
            pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
            script = soup.find("script", text=pattern)
            data = pattern.search(script.text).group(1)
            data = json.loads(data)
            if data:
                for row in data["data"]["artists"]: 
                    for k,v in row.items():
                        row.update({k:str(v)})
                    #results.append(row)
                    file.write(', '.join(row.values()) + '\n')
            else:
                break
    return results

output = open('output/band_list.csv', 'w+') 
headers = ['id', 'name', 'tabscount', 'artisct_url', 'tabs_last_update_timestamp']
output.write(', '.join(headers) + '\n')
#pasing files
datadir = 'data/'
data = getList(output)
output.close()
    


    
#https://www.ultimate-guitar.com/bands/a.htm

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json
import time
from selenium import webdriver
from pathlib import Path


seperator = ', '
def getBandTree(band, page):
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%4B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&view_state=advanced&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
    return BeautifulSoup(pageBand.content)
 
def download(browser, id, i = 0):
    browser.get("https://tabs.ultimate-guitar.com/download/public/" + id)
    time.sleep(1)
       
    if i == 0:    
        # click popup
        time.sleep(2)
        doc = browser.find_elements_by_xpath('/html/body/div[3]/div/div/div/div/footer/button[1]')[0]
        doc.click()
    
    doc = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[2]/main/div[2]/div[2]/article/div[1]/section[2]/form/div/button')[0]
    doc.click()

    
def get_chrome_driver(download_dir):
    chrome_options = webdriver.ChromeOptions()
    prefs = {
      "download.default_directory": download_dir,
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True
    }
    chrome_options.add_experimental_option("prefs", prefs)

    return webdriver.Chrome(options=chrome_options)    
    
def main():
    band = 'kreator'
    curr_dir = Path.cwd()
    download_dir = str(curr_dir) + "/data/" + band
    
    page = 1
    bandURL = getBandTree(band, page)
    ids = []
    #Get max pages
    pages = bandURL.find_all('div', { "class" : "paging" })
    maxPages = str(pages).count('')
    dfs = []
    for page in range(1, maxPages + 1):
        soup = getBandTree(band, page)
        pattern = re.compile(r"window.UGAPP.store.page\s+=\s+(\{.*?\});\n")
        script = soup.find("script", text=pattern)

        data = pattern.search(script.text).group(1)
        data = json.loads(data)
        for row in data["data"]["results"]: #type = Pro
            #if 'ktype' in row.keys():
            pos = row["tab_url"].find("guitar_pro_")
            if pos > 0:
                id = row["tab_url"][pos:].replace('guitar_pro_', '')
                ids.append(id)
                #download(id)

    browser = get_chrome_driver(download_dir)    
    
    i = 0
    for id in ids:
        print(id)
        #download(browser, id, i)
        i = i + 1
    #browser.quit()
        
if __name__=='__main__':
    main()
    
#https://www.ultimate-guitar.com/artist/exodus_4988?filter=guitar_pro

1920599
223489
472130
950360
1019611
1774268
662401
950417
255927
393412
541296
649057
942159
1102090
1106208
1852797
223500
723967
885298
223512
223513
223514
1165375
223493
1905253
2426757
700749
941481
223497
572722
1162332
2043023
2106163
1760526
938672
2420985
1540219
1083431
947128
472128
1152810
1206851
1213164
885292
1806556
1801438
2006795
2010115
576792
577223
762249
941158
954519
1681800
2255231
946831
2321353
223488
650313
398617
472126
922075
966097
1507850
1736843
647518
962441
1099894
1071161
1083678
223490
885297
1207198
2048576
223491
1743067
1680308
1770286
1866763
1759026
1033831
472127
821486
1765722
1051152
960777
223492
499070
949439
649016
922777
908449
668136
1433545
1508982
255926
976556
792852
799616
821488
223494
416963
942937
478143
1725560
487985
1085410
1677757
1235809
1513219
223495
223496
1879045
812239
2254145
255924
1143845
1426755
223498
223499
478406
807376
926097
361068
949859
961045
223501
1102472
223502
533527
223503
789090
1141446
2850773
728353
